In [2]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

## Step 1 Data Preprocesessing & Feature Engineering

_Randomly distribute the ratio for R&D Spend, Administration spend, Marketing spend, all can drive(total expenses * ratio) from the total expenses. 

_Company Founded Longevity : How long the company has been founded from current years (year founded - current year)
Total Growth 

_Total Growth (Total rev 2019- Total rev in 2017 / Total rev in 2017)

_Growth rate in last 3 years (Total growth/ past 2 years)


In [3]:
dt_financials = pd.DataFrame(pd.read_excel(open('../dataset/P11-1000-Startups.xlsx', 'rb'), 
                                           sheet_name = "Financials", ingnore_index= True), index = None)
dt_financials.set_index(['ID'])
dt_overview = pd.DataFrame(pd.read_excel(open('../dataset/P11-1000-Startups.xlsx', 'rb'), 
                                              sheet_name = "Overview", ingnore_index= True), index = None)
dt_overview.set_index(['ID'])
# merge the two tab toagther by common keys ID and Name
new_dt = pd.merge(dt_financials, dt_overview,   on=['ID', "Name"])

In [5]:
# create Total growth
total_growth = list(round((new_dt['2019 Revenue']-new_dt['2017 Revenue'])/new_dt['2017 Revenue'], 2))
new_dt["Total Growth %"] = total_growth
# generate avg Rev, expenses, profit
new_dt["Avg Revenue (2017-2019)"] = np.round((new_dt["2019 Revenue"] +new_dt["2018 Revenue"] +
                                              new_dt["2017 Revenue"]) /3, 2)
new_dt["Avg Expenses (2017-2019)"] = np.round((new_dt["2019 Expenses"] +new_dt["2018 Expenses"] +
                                              new_dt["2017 Expenses"]) /3, 2)
new_dt["Avg Profit (2017-2019)"] = np.round((new_dt["2019 Profit"] + new_dt["2018 Profit"] +
                                              new_dt["2017 Profit"]) /3, 2)

In [6]:
# create Company Founded Longevity
new_dt['Year Founded'].astype(int)
founded_longevity = []
for yr in list(new_dt['Year Founded']):
    longev = 2020-yr
    if longev == 0:
        longev = 1
    founded_longevity.append(longev)
new_dt['Company Founded Longevity'] = founded_longevity


In [7]:
# generate column: Growth rate in last 3 years
last3yr_growth_rate = new_dt['Total Growth %']/2
new_dt['Growth rate in Last 3 Years in %'] = last3yr_growth_rate

In [8]:
# Randomly distribute the ratio for R&D Spend, Administration spend, Marketing spend, 
# all can drive(total expenses * ratio) from the total expenses.
num_ratio = 1000
ratio_marketing =[round(float(str(random.randrange(20, 35)) + '.' + str(random.randrange(0, 100)))/100, 4) for i in range(num_ratio) ]
ratio_admin =[round(float(str(random.randrange(14, 30)) + '.' + str(random.randrange(0, 100)))/100, 4) for i in range(num_ratio) ]
ratio_Rd  =[1- ratio_marketing[i] - ratio_admin[i] for i in range(num_ratio)]


new_dt["Admin Spend (2019) Ratio"] = ratio_admin
new_dt["Marketing & Sales Spend (2019) Ratio"] = ratio_marketing
new_dt["R&D Spend (2019) Ratio"] = ratio_Rd 

new_dt["Admin Spend (2019)"] = np.round(new_dt["2019 Expenses"] * new_dt["Admin Spend (2019) Ratio"], 3)
new_dt["Marketing & Sales Spend (2019)"] = np.round(new_dt["2019 Expenses"] * new_dt["Marketing & Sales Spend (2019) Ratio"], 3)
new_dt["R&D Spend (2019)"] = np.round(new_dt["2019 Expenses"] * new_dt["R&D Spend (2019) Ratio"] , 3)
new_dt.head()

,ID,Name,2019 Revenue,2019 Expenses,2019 Profit,2019 Growth %,2018 Revenue,2018 Expenses,2018 Profit,2018 Growth %,...,Avg Expenses (2017-2019),Avg Profit (2017-2019),Company Founded Longevity,Growth rate in Last 3 Years in %,Admin Spend (2019) Ratio,Marketing & Sales Spend (2019) Ratio,R&D Spend (2019) Ratio,Admin Spend (2019),Marketing & Sales Spend (2019),R&D Spend (2019)
0,1,Over-Hex,10684526.90,1130700.06,9553826.84,3.34,2463739,194747.86,2268991.14,4.10,...,456280.79,4087503.21,10,10.560,0.2796,0.2783,0.4421,316143.737,314673.827,499882.497
1,2,Unimattax,9016542.89,804035.40,8212507.49,0.65,5448571,239226.90,5209344.10,0.41,...,378325.58,5731457.30,7,0.665,0.1948,0.2598,0.5454,156626.096,208888.397,438520.907
2,3,Lexila,12456641.64,2149297.00,10307344.65,5.20,2007778,335126.93,1672651.07,4.23,...,865560.45,4083878.23,3,15.725,0.1594,0.3238,0.5168,342597.942,695942.369,1110756.690
3,4,Greenfax,7746272.07,1044375.49,6701896.58,3.17,1857830,247048.76,1610781.24,0.56,...,638955.64,2959383.94,4,2.750,0.2928,0.3227,0.3845,305793.143,337019.971,401562.376
4,5,Saoace,10725585.46,2694644.42,8030941.04,3.12,2604017,615040.85,1988976.15,2.39,...,1177549.38,3521700.31,7,6.480,0.2896,0.3300,0.3804,780369.024,889232.659,1025042.737


In [9]:
new_dt.to_csv("../dataset/P11-1000-Startups_cleaned.csv", index = None)

### group similer industries into one and export into another csv for later modeling analysis

for exmaple: Information technology, Computer hardware, IT Service, Software

## Generate Exploratory Analysis DataSet by Industry

In [63]:
# this will creat a multi level columns
df_industry_agg = new_dt.groupby('Industry').agg({
    "Name": 'count',
    "2019 Revenue": [np.mean, 'median'],
    "2019 Expenses": [np.mean, 'median'],
    "2019 Growth %": [np.mean, 'median']
})
df_industry_agg.head(2)

# list(df_industry_agg.columns.levels[0])
# list(df_industry_agg.columns.levels[1])

Name  2019 Revenue             2019 Expenses  \
                             count          mean      median          mean   
Industry                                                                     
Advertising & Marketing         65  9.142556e+06  9145449.94  4.444607e+06   
Business Products & Services    67  8.889641e+06  8836311.51  4.244925e+06   

                                         2019 Growth %         
                                  median          mean median  
Industry                                                       
Advertising & Marketing       4515537.82      4.032769   3.92  
Business Products & Services  3895860.23      4.033582   3.90

In [81]:
columns_lst = ["Industry", "Company Count", "2019 Revenue Mean", "2019 Expenses Mean", "2019 Growth % Mean", 
           "2019 Revenue Median", "2019 Expenses Median", "2019 Growth % Median"]
df_industry = pd.DataFrame(columns = columns_lst)
df_industry["2019 Revenue Mean"]= list(np.round(df_industry_agg["2019 Revenue"]["mean"], 2))
df_industry["Industry"]= list(df_industry_agg.index)
df_industry["Company Count"] = list(df_industry_agg["Name"]["count"])
df_industry["2019 Revenue Mean"] = list(np.round(df_industry_agg["2019 Revenue"]["mean"], 2))
df_industry["2019 Expenses Mean"]= list(np.round(df_industry_agg["2019 Expenses"]["mean"], 2))
df_industry["2019 Growth % Mean"]= list(np.round(df_industry_agg["2019 Growth %"]["mean"], 2))
df_industry["2019 Revenue Median"]=list(np.round(df_industry_agg["2019 Revenue"]["median"], 2))
df_industry["2019 Expenses Median"]= list(np.round(df_industry_agg["2019 Expenses"]["median"], 2))
df_industry["2019 Growth % Median"]= list(np.round(df_industry_agg["2019 Growth %"]["median"], 2))
df_industry.to_csv('../dataset/P11-1000-Startups_industry_analysis.csv')

## Step 2 Tableau Analysis Reporting

##### Growth Analysis Dashbaord
- growth data to display top growth firm
- scatter plot: deep green or greens are counted as top growing firm with the best performing revenue and expense
Dashboard Link: https://public.tableau.com/profile/xiaoyang.wu#!/vizhome/StartupTopGrowthAnalysis/TopGrowthAnalysis?publish=yes

![dashboard1](../Img/Dashboard1.png)

#### Selected Potential Companies for Investment Dashboard
- Selected Companies Analysis
- Selected companies Industry analysis (bar chart)
Dashboard Link: https://public.tableau.com/profile/xiaoyang.wu#!/vizhome/SelectedStartupforVentureCapitalInvestingAnalysis/SelectedPotentialCompaniesforInvestment?publish=yes

![dashboard2](../Img/Dashboard2.png)

##### Top Profitable Companies to Invest Dashboard
- Top N Most Profitable Companies (Portfolio Analysis)
- Top N Mots Profitable Companies (tree map)
Dashboard Link: https://public.tableau.com/profile/xiaoyang.wu#!/vizhome/TopNMostProfitableStartupforInvestingAnalysis/TopProfitableCompaniestoInvest?publish=yes

![dashboard3](../Img/Dashboard3.png)


## Step 3 Build a Pediction Model Based Selected 100 Companies 

#### Model 1: Multiple Linear Regression

In [ ]:
# data import 

In [ ]:
# data ppc

In [ ]:
# dummy variables ppc

In [ ]:
# split train and test data

In [ ]:
# train data

In [ ]:
# test data

In [ ]:
# compare result

#### Model 2: Logistic Regression 